In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from esda import Moran
import geopandas as gpd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from libpysal.weights import KNN
from spreg import OLS, GM_Error_Het
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [20]:
gdf = gpd.read_file('output/hedonic_gdf.gpkg')

In [21]:
gdf.head()

,GrossSalePrice,AgeAtSale,LandArea,TotalFloorArea,water_DIST,bus_DIST,Census_Pop,RnkIMDNoEm,RnkIMDNoIn,RnkIMDNoCr,...,Median_Income,CBD_DIST,cycleways_DIST,cycle_DENS,canopy_0_20,canopy_20_50,canopy_50_100,canopy_100_200,residuals,geometry
0,610000,24,685.0,211.0,189.601703,216.798910,930.0,1811.0,849.0,1231.0,...,79353,6354.291389,590.000000,1.6,1.077267,163.500474,3673.623939,10193.360430,0.038369,POINT (1564919.129 5174157.305)
1,477500,42,609.0,120.0,139.581200,154.824709,930.0,1811.0,849.0,1231.0,...,79353,6227.397408,471.071045,1.6,20.957896,450.415001,3568.941048,19805.408025,0.071420,POINT (1565047.92 5174178.753)
2,655000,22,704.0,245.0,201.942495,192.941029,930.0,1811.0,849.0,1231.0,...,79353,6352.891850,610.836121,1.6,10.299413,144.270775,2970.792200,9872.392485,0.009830,POINT (1564904.931 5174179.536)
3,500000,44,869.0,131.0,157.854014,179.645498,930.0,1811.0,849.0,1231.0,...,79353,6229.065719,483.016235,1.6,74.306779,392.823676,3602.928325,19098.845813,0.079604,POINT (1565029.188 5174201.753)
4,880000,22,1149.0,279.0,199.693219,216.730600,930.0,1811.0,849.0,1231.0,...,79353,6258.811869,533.936462,1.6,0.000000,279.813206,6163.843751,14113.518330,0.190113,POINT (1564988.184 5174211.698)


In [22]:
gdf['log_price'] = np.log(gdf['GrossSalePrice'])

In [23]:
gdf.columns

Index(['GrossSalePrice', 'AgeAtSale', 'LandArea', 'TotalFloorArea',
       'water_DIST', 'bus_DIST', 'Census_Pop', 'RnkIMDNoEm', 'RnkIMDNoIn',
       'RnkIMDNoCr', 'RnkIMDNoHo', 'RnkIMDNoHe', 'RnkIMDNoEd', 'RnkIMDNoAc',
       'DECILE_high', 'DECILE_prim', 'Median_Income', 'CBD_DIST',
       'cycleways_DIST', 'cycle_DENS', 'canopy_0_20', 'canopy_20_50',
       'canopy_50_100', 'canopy_100_200', 'residuals', 'geometry',
       'log_price'],
      dtype='object')

In [24]:
w = KNN.from_dataframe(gdf, k=8)
w.transform = 'R'

/Users/rujalshrestha/Projects/chc-property-prices/venv/lib/python3.13/site-packages/libpysal/weights/distance.py:153: UserWarning: The weights matrix is not fully connected: 
 There are 11 disconnected components.
  W.__init__(self, neighbors, id_order=ids, **kwargs)


In [25]:
gdf.shape

(12431, 27)

In [26]:
y = gdf['log_price'].values.reshape(-1, 1)

non_x_cols = [
  'GrossSalePrice', 'residuals', 'geometry', 'log_price', # unneccessary columns
  "Census_Pop", "RnkIMDNoEm", "RnkIMDNoIn", "RnkIMDNoCr", "RnkIMDNoHo", "RnkIMDNoHe", "RnkIMDNoEd", "RnkIMDNoAc", "DECILE_high", "Median_Income" # removed columns
]

x_cols = [col for col in gdf.columns if col not in non_x_cols]

print(x_cols)

X = gdf.loc[:, x_cols]

X = X.values

['AgeAtSale', 'LandArea', 'TotalFloorArea', 'water_DIST', 'bus_DIST', 'DECILE_prim', 'CBD_DIST', 'cycleways_DIST', 'cycle_DENS', 'canopy_0_20', 'canopy_20_50', 'canopy_50_100', 'canopy_100_200']


In [36]:
ols_spreg = OLS(y, X, w=w, spat_diag=True, moran=True, name_y='log_price', name_x=x_cols)

print(ols_spreg.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   log_price                Number of Observations:       12431
Mean dependent var  :     13.1505                Number of Variables   :          14
S.D. dependent var  :      0.3376                Degrees of Freedom    :       12417
R-squared           :      0.5908
Adjusted R-squared  :      0.5904
Sum squared residual:     579.576                F-statistic           :   1379.1601
Sigma-square        :       0.047                Prob(F-statistic)     :           0
S.E. of regression  :       0.216                Log likelihood        :    1415.730
Sigma-square ML     :       0.047                Akaike info criterion :   -2803.459
S.E of regression ML:      0.2159                Schwarz criterion     :   -2699.468

------------------------------------------------------------

# VIF

In [28]:
X = gdf.loc[:, x_cols]

In [29]:
X.head()

,AgeAtSale,LandArea,TotalFloorArea,water_DIST,bus_DIST,DECILE_prim,CBD_DIST,cycleways_DIST,cycle_DENS,canopy_0_20,canopy_20_50,canopy_50_100,canopy_100_200
0,24,685.0,211.0,189.601703,216.798910,10.0,6354.291389,590.000000,1.6,1.077267,163.500474,3673.623939,10193.360430
1,42,609.0,120.0,139.581200,154.824709,10.0,6227.397408,471.071045,1.6,20.957896,450.415001,3568.941048,19805.408025
2,22,704.0,245.0,201.942495,192.941029,10.0,6352.891850,610.836121,1.6,10.299413,144.270775,2970.792200,9872.392485
3,44,869.0,131.0,157.854014,179.645498,10.0,6229.065719,483.016235,1.6,74.306779,392.823676,3602.928325,19098.845813
4,22,1149.0,279.0,199.693219,216.730600,10.0,6258.811869,533.936462,1.6,0.000000,279.813206,6163.843751,14113.518330


In [30]:
vif_df = pd.DataFrame({
  'variable': X.columns,
  'VIF': [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
})

print(vif_df)

          variable       VIF
0        AgeAtSale  3.421698
1         LandArea  1.189728
2   TotalFloorArea  5.826208
3       water_DIST  2.510987
4         bus_DIST  2.777976
5      DECILE_prim  1.514794
6         CBD_DIST  6.041276
7   cycleways_DIST  3.846305
8       cycle_DENS  1.983214
9      canopy_0_20  2.980776
10    canopy_20_50  5.595189
11   canopy_50_100  2.576094
12  canopy_100_200  3.622587


# training GM_Error_Het heteroskedasticity-robust estimation

In [31]:
X = X.values

In [32]:
sem = GM_Error_Het(y, X, w, name_y = 'log_price', name_x = x_cols)

In [33]:
print(sem.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: GM SPATIALLY WEIGHTED LEAST SQUARES (HET)
------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   log_price                Number of Observations:       12431
Mean dependent var  :     13.1505                Number of Variables   :          14
S.D. dependent var  :      0.3376                Degrees of Freedom    :       12417
Pseudo R-squared    :      0.5839
N. of iterations    :           1                Step1c computed       :          No

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        12.79764         0.02965       431.55548         0.00000
           AgeAtSale        -0.00174         0.00012    

In [34]:
gdf['u_sem'] = sem.u
mi_sem = Moran(gdf['u_sem'], w)
print(mi_sem.I, mi_sem.p_sim)

0.5305858446370211 0.001


In [35]:
gdf["e_sem"] = sem.e_filtered.flatten()  # or sem.e_filtered if already 1D
mi_sem = Moran(gdf["e_sem"], w)
print(mi_sem.I, mi_sem.p_sim)

-0.017583733154981365 0.001
